In [54]:
import random
import numpy as np
from data_process import get_CIFAR10_data
%matplotlib inline
from save_submission import output_submission_csv

# Loading CIFAR-10

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 50000]
, TEST_IMAGES = 10000

In [55]:
# You can change these numbers for experimentation
# For submission we will use the default values 

#Playing arounf with the values in-order to study the dataset
# TRAIN_IMAGES = 4
# VAL_IMAGES = 10


TRAIN_IMAGES = 40000
VAL_IMAGES = 10000

In [56]:
data = get_CIFAR10_data(TRAIN_IMAGES, VAL_IMAGES)
X_train_CIFAR, y_train_CIFAR = data['X_train'], data['y_train']
X_val_CIFAR, y_val_CIFAR = data['X_val'], data['y_val']
X_test_CIFAR, y_test_CIFAR = data['X_test'], data['y_test']
n_class_CIFAR = len(np.unique(y_test_CIFAR))

In [57]:
print(X_train_CIFAR)


[[[[-7.18991250e+01 -8.72976000e+01 -8.12357000e+01 ...  2.67462500e+01
     2.14346500e+01  1.76517250e+01]
   [-1.14228675e+02 -1.29459550e+02 -1.12339025e+02 ... -7.25522500e+00
    -1.05204500e+01 -7.31355000e+00]
   [-1.04823175e+02 -1.12813825e+02 -8.06145500e+01 ... -1.13619750e+01
    -8.75937500e+00 -1.97586000e+01]
   ...
   [ 8.13622000e+01  7.63019000e+01  7.40273750e+01 ...  3.55747000e+01
    -6.88562250e+01 -7.26718250e+01]
   [ 5.28395750e+01  4.74419750e+01  6.06504750e+01 ...  5.82546000e+01
    -2.88197500e+01 -4.31803750e+01]
   [ 4.91837000e+01  4.13473750e+01  5.20684000e+01 ...  8.89515250e+01
     2.41888500e+01 -3.86372500e+00]]

  [[-7.41391000e+01 -8.94913250e+01 -8.83168250e+01 ... -4.35872500e+00
    -1.07493000e+01 -1.15151000e+01]
   [-1.15307550e+02 -1.34451950e+02 -1.27177225e+02 ... -4.72118500e+01
    -5.16138250e+01 -4.74037250e+01]
   [-1.10622775e+02 -1.26470025e+02 -1.07078775e+02 ... -4.99957250e+01
    -4.95833250e+01 -6.05901000e+01]
   ...
   

In [58]:
print(X_train_CIFAR.shape)

(40000, 3, 32, 32)


Convert the sets of images from dimensions of **(N, 3, 32, 32) -> (N, 3072)** where N is the number of images so that each **3x32x32** image is represented by a single vector.

In [59]:
X_train_CIFAR = np.reshape(X_train_CIFAR, (X_train_CIFAR.shape[0], -1))
X_val_CIFAR = np.reshape(X_val_CIFAR, (X_val_CIFAR.shape[0], -1))
X_test_CIFAR = np.reshape(X_test_CIFAR, (X_test_CIFAR.shape[0], -1))

In [60]:
print(X_train_CIFAR)

[[-71.899125 -87.2976   -81.2357   ...  25.958275 -30.104    -42.5743  ]
 [ 23.100875  -4.2976   -26.2357   ...  24.958275  27.896     29.4257  ]
 [124.100875 122.7024   121.7643   ... -31.041725 -31.104    -30.5743  ]
 ...
 [ 36.100875  33.7024    19.7643   ...  17.958275  27.896     15.4257  ]
 [-85.899125 -84.2976   -85.2357   ... -84.041725 -80.104    -90.5743  ]
 [104.100875 108.7024   107.7643   ...  67.958275  60.896     59.4257  ]]


In [61]:
print(X_train_CIFAR[0])

[-71.899125 -87.2976   -81.2357   ...  25.958275 -30.104    -42.5743  ]


In [62]:
print(X_train_CIFAR.shape)

(40000, 3072)


# Get Accuracy
This function computes how well your model performs using accuracy as a metric.


In [63]:
def get_acc(pred, y_test):
    return np.sum(y_test==pred)/len(y_test)*100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.


The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


# Model Perceptron

In [36]:
import numpy as np

class Perceptron:
    def __init__(self, n_class: int, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            n_class: the number of classes
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.Weights = None
        self.lr = lr
        self.epochs = epochs
        self.n_class = n_class
        self.activation_func = self.helper_function

    def helper_function(self, x):
        return np.where(x>=0, 1, 0)
        
    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the perceptron update rule as introduced in Lecture 3.
        Parameters:
            X_train: a number array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        
        ###### YOUR CODE STARTS HERE ######
        N, D = X_train.shape
        self.Weights = np.zeros((D+1,))
        X_train = np.concatenate([X_train, np.ones((N, 1))], axis = 1)
        for i in range(self.epochs):
            for j in range(N):
                if y_train[j] * np.dot(self.Weights, X_train[j, :]) <= 0:
                    self.Weights += y_train[j] * X_train[j, :]
        ###### YOUR CODE ENDS HERE ######

    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """

        ###### YOUR CODE STARTS HERE ######
        n_samples = X_test.shape[0]
        X_test = np.concatenate([X_test, np.ones((n_samples, 1))], axis=1)
        y_test = np.matmul(X_test, self.Weights)
        y_test = np.vectorize(lambda val: 1 if val > 0 else -1)(y_test)
        ###### YOUR CODE ENDS HERE ######
        return y_test


## Train Perceptron on CIFAR

In [37]:
lr = 0.5
n_epochs = 50
n_class_CIFAR = 10

percept_CIFAR = Perceptron(n_class_CIFAR, lr, n_epochs)
percept_CIFAR.train(X_train_CIFAR, y_train_CIFAR)

In [38]:
pred_percept = percept_CIFAR.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_CIFAR)))

The training accuracy is given by: 4.657500


### Validate Perceptron on CIFAR

In [73]:
pred_percept = percept_CIFAR.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_CIFAR)))

The validation accuracy is given by: 5.010000


### Test Perceptron on CIFAR

In [72]:
pred_percept = percept_CIFAR.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_CIFAR)))

The testing accuracy is given by: 4.980000


In [71]:
output_submission_csv('output/Logistic_submission_CIFAR.csv', percept_CIFAR.predict(X_test_CIFAR))

# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.




The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

In [70]:
"""Logistic regression model."""

import numpy as np
from numpy import log, dot
import math


class Logistic:
    def __init__(self, lr: float, epochs: int):
        """Initialize a new classifier.
        Parameters:
            lr: the learning rate
            epochs: the number of epochs to train for
        """
        self.Weights = 1
        self.lr = lr
        self.epochs = epochs

    def sigmoid(self, z: np.ndarray) -> np.ndarray:
        """Sigmoid function.
        Parameters:
            z: the input
        Returns:
            the sigmoid of the input
        """
        ##### YOUR CODE STARTS HERE #####
        ret = 1/(1+np.exp(-z))
        
        ##### YOUR CODE ENDS HERE #####
        return ret

    def cost_function(self, X, y, weights):
        dot_z = dot(X, weights)
        pred_sample_0 = y * log(self.sigmoid(dot_z))
        pred_sample_2 = (1-y) * log(1-self.sigmoid(dot_z))
        return - sum(pred_sample_0 + pred_sample_2)/ len(X)

    def train(self, X_train: np.ndarray, y_train: np.ndarray):
        """Train the classifier.
        Use the logistic regression update rule as introduced in lecture.
        Parameters:
            X_train: a numpy array of shape (N, D) containing training data;
                N examples with D dimensions
            y_train: a numpy array of shape (N,) containing training labels
        """
        N, D = X_train.shape
        loss = []
        w = np.random.randn(X_train.shape[1])
        #self.Weights = np.random.randn(1, D)
        N = len(X_train)

        ##### YOUR CODE STARTS HERE #####
        
        for i in range(self.epochs):
            y_hat = self.sigmoid(dot(X_train, w))
            w -= self.lr*dot(X_train.T, y_hat-y_train)/N
            loss.append(self.cost_function(X_train, y_train, w))

        self.Weights=w
        self.loss=loss 
        ##### YOUR CODE ENDS HERE #####
        
        
    def predict(self, X_test: np.ndarray) -> np.ndarray:
        """Use the trained weights to predict labels for test data points.
        Parameters:
            X_test: a numpy array of shape (N, D) containing testing data;
                N examples with D dimensions
        Returns:
            predicted labels for the data in X_test; a 1-dimensional array of
                length N, where each element is an integer giving the predicted
                class.
        """
        N, D = X_test.shape
        y_test = np.zeros(N)
        ##### YOUR CODE STARTS HERE #####
        dot_p_z = dot(X_test, self.Weights)
        y_test=[1 if i >0.7 else 0 for i in self.sigmoid(dot_p_z)]
        ##### YOUR CODE ENDS HERE #####
        return y_test

### Training Logistic Classifer

In [66]:
learning_rate =0.6
n_epochs =50

lr = Logistic(learning_rate, n_epochs)
lr.train(X_train_CIFAR, y_train_CIFAR)

/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:27: RuntimeWarning: overflow encountered in exp
  ret = 1/(1+np.exp(-z))
/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:34: RuntimeWarning: divide by zero encountered in log
  pred_sample_0 = y * log(self.sigmoid(dot_z))
/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:34: RuntimeWarning: invalid value encountered in multiply
  pred_sample_0 = y * log(self.sigmoid(dot_z))
/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:35: RuntimeWarning: divide by zero encountered in log
  pred_sample_2 = (1-y) * log(1-self.sigmoid(dot_z))
/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:35: RuntimeWarning: invalid value encountered in multiply
  pred_sample_2 = (1-y) * log(1-self.sigmoid(dot_z))
/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:36: RuntimeWarning: invalid value e

In [67]:
pred_lr = lr.predict(X_train_CIFAR)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_CIFAR)))

The training accuracy is given by: 12.142500


/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:27: RuntimeWarning: overflow encountered in exp
  ret = 1/(1+np.exp(-z))


### Validate Logistic Classifer

In [68]:
pred_lr = lr.predict(X_val_CIFAR)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_CIFAR)))

The validation accuracy is given by: 12.170000


/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:27: RuntimeWarning: overflow encountered in exp
  ret = 1/(1+np.exp(-z))


### Test Logistic Classifier

In [69]:
pred_lr = lr.predict(X_test_CIFAR)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_CIFAR)))
output_submission_csv('output/Logistic_submission_CIFAR.csv', lr.predict(X_test_CIFAR))

/var/folders/fw/cxj36c7j5t9g1vd7yw1sdtxh0000gn/T/ipykernel_9977/3858848026.py:27: RuntimeWarning: overflow encountered in exp
  ret = 1/(1+np.exp(-z))


The testing accuracy is given by: 12.420000
